# Basics of Data Management

This notebook follows the **Basics of Data Management** slides and implements the examples in both **R** and **Python** using Medicare Advantage data snippets for Georgia.

## Setup: R in Colab

First, enable R in this Colab environment using `rpy2`. Run this cell once at the top of the notebook.

In [2]:
!pip -q install rpy2
%load_ext rpy2.ipython


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


## Setup: R packages

Install and load the R packages we will use. You only need to install once; on later runs, `pacman::p_load()` will just load them.

In [3]:
%%R
options(repos = c(CRAN = "https://cloud.r-project.org"))

# Install pacman if it is not already installed
if (!"pacman" %in% rownames(installed.packages())) {
  install.packages("pacman")
}

library(pacman)

# Load the packages you want
p_load(tidyverse, janitor, lubridate, readr)

print("R is ready in Colab ✅")


[1] "R is ready in Colab <U+2705>"


## Loading the Georgia Medicare Advantage snippets (R)

We will read in three CSV files hosted on GitHub: enrollment, contracts, and service areas.

In [4]:
%%R
base_url <- "https://raw.githubusercontent.com/imccart/empirical-methods-content/main/data/output/ma-snippets/"

ga_enrollment <- read_csv(paste0(base_url, "ga-enrollment.csv"))
ga_contract   <- read_csv(paste0(base_url, "ga-contract.csv"))
ga_service    <- read_csv(paste0(base_url, "ga-service-area.csv"))

glue::glue("ga_enrollment: {nrow(ga_enrollment)} rows, {ncol(ga_enrollment)} cols")


Rows: 7333 Columns: 7
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (3): contractid, state, county
dbl (4): planid, ssa, fips, enrollment

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 172 Columns: 12
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (11): contractid, org_type, plan_type, partd, snp, eghp, org_name, org_m...
dbl  (1): planid

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 4012 Columns: 11
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (6): contractid, org_name, org_type, plan_type, county, state
dbl (2): ssa, fips
lgl (3): partial, eghp, notes

i Use `spec()` to retrieve the full column speci

## Setup: Python imports

Now set up the Python side with `pandas` (and `numpy` for a few transformations).

In [5]:
import pandas as pd
import numpy as np

print("Python is ready in Colab ✅")


Python is ready in Colab ✅


## Loading the Georgia Medicare Advantage snippets (Python)

We read the same CSVs directly from GitHub using `pandas.read_csv()`.

In [6]:
base_url = "https://raw.githubusercontent.com/imccart/empirical-methods-content/main/data/output/ma-snippets/"

ga_enrollment = pd.read_csv(base_url + "ga-enrollment.csv")
ga_contract   = pd.read_csv(base_url + "ga-contract.csv")
ga_service    = pd.read_csv(base_url + "ga-service-area.csv")

ga_enrollment.shape, ga_contract.shape, ga_service.shape


((7333, 7), (172, 12), (4012, 11))

# Looking at your data

We start by doing basic checks: object type, dimensions, and the first few rows.

### First checks after loading (R)

In [7]:
%%R
# Basic info about ga_enrollment
class(ga_enrollment)
dim(ga_enrollment)
nrow(ga_enrollment)
ncol(ga_enrollment)

# Peek at the data
head(ga_enrollment)


# A tibble: 6 x 7
  contractid planid   ssa  fips state county   enrollment
  <chr>       <dbl> <dbl> <dbl> <chr> <chr>         <dbl>
1 H0111           1 11000 13001 GA    Appling          11
2 H0111           1 11030 13009 GA    Baldwin          27
3 H0111           1 11050 13013 GA    Barrow           47
4 H0111           1 11060 13015 GA    Bartow           43
5 H0111           1 11070 13017 GA    Ben Hill         14
6 H0111           1 11080 13019 GA    Berrien          13


### First checks after loading (Python)

In [8]:
# Basic info about ga_enrollment
type(ga_enrollment)
ga_enrollment.shape
ga_enrollment.shape[0]   # rows
ga_enrollment.shape[1]   # columns

# Peek at the data
ga_enrollment.head()


,contractid,planid,ssa,fips,state,county,enrollment
0,H0111,1,11000,13001,GA,Appling,11
1,H0111,1,11030,13009,GA,Baldwin,27
2,H0111,1,11050,13013,GA,Barrow,47
3,H0111,1,11060,13015,GA,Bartow,43
4,H0111,1,11070,13017,GA,Ben Hill,14


### Structure and summaries

Next, look at variable names, types, and basic numeric summaries.

#### R

In [ ]:
%%R
library(dplyr)

# Structure
glimpse(ga_enrollment)

# Basic numeric summaries
summary(ga_enrollment)


#### Python

In [ ]:
# Structure (column names and types)
ga_enrollment.info()

# Basic numeric summaries
ga_enrollment.describe()


### What to look for

- **Ranges and typical values**: negative ages, impossible years, absurd enrollment counts
- **Missingness**: are key variables mostly missing? do some years/counties have more missing than others?
- **Outliers**: a few very large or very small values that might be data errors
- **Types**: are IDs stored as strings? are dates actually dates?


### Simple summaries and counts

We can quickly summarise enrollment by year and other groups.

#### R

In [ ]:
%%R
library(dplyr)

# How many plans per year?
ga_enrollment |>
  count(year)

# Average enrollment by year
ga_enrollment |>
  group_by(year) |>
  summarise(
    avg_enrollment = mean(enrollment, na.rm = TRUE),
    n_plans        = n(),
    .groups        = "drop"
  )


#### Python

In [ ]:
# How many plans per year?
ga_enrollment.groupby("year").size()

# Average enrollment by year
ga_enrollment.groupby("year").agg(
    avg_enrollment=("enrollment", "mean"),
    n_plans=("enrollment", "size")
)


# Cleaning and managing your data

Now we start selecting variables, transforming them, and handling missing or implausible values.

### Selecting variables and filtering rows (R)

In [ ]:
%%R
library(dplyr)

ga_enrollment_small <- ga_enrollment |>
  select(year, contract_id, plan_id, county, enrollment) |>
  filter(year >= 2015, enrollment > 0)

glimpse(ga_enrollment_small)


### Selecting variables and filtering rows (Python)

In [ ]:
ga_enrollment_small = (
    ga_enrollment
      [["year", "contract_id", "plan_id", "county", "enrollment"]]
      .loc[(ga_enrollment["year"] >= 2015) & (ga_enrollment["enrollment"] > 0)]
)

ga_enrollment_small.head()


### Creating and transforming variables (R)

In [ ]:
%%R
ga_enrollment_clean <- ga_enrollment_small |>
  mutate(
    log_enrollment = log(enrollment),
    rural          = if_else(county_type == "Rural", 1L, 0L),
    year_factor    = factor(year)
  )

glimpse(ga_enrollment_clean)


### Creating and transforming variables (Python)

In [ ]:
ga_enrollment_clean = ga_enrollment_small.copy()
ga_enrollment_clean["log_enrollment"] = np.log(ga_enrollment_clean["enrollment"])
ga_enrollment_clean["rural"] = (ga_enrollment_clean["county_type"] == "Rural").astype(int)
ga_enrollment_clean["year_factor"] = ga_enrollment_clean["year"].astype("category")

ga_enrollment_clean.head()


### Handling missing and implausible values (R)

In [ ]:
%%R
ga_enrollment_checked <- ga_enrollment_clean |>
  mutate(
    enrollment = if_else(enrollment < 0, NA_real_, enrollment)
  ) |>
  filter(!is.na(enrollment), !is.na(contract_id))

glimpse(ga_enrollment_checked)


### Handling missing and implausible values (Python)

In [ ]:
ga_enrollment_checked = ga_enrollment_clean.copy()

# Recode negative enrollment as missing
ga_enrollment_checked.loc[
    ga_enrollment_checked["enrollment"] < 0,
    "enrollment"
] = pd.NA

# Drop rows missing key fields
ga_enrollment_checked = ga_enrollment_checked.dropna(
    subset=["enrollment", "contract_id"]
)

ga_enrollment_checked.head()


# Merging and reshaping data

We now combine information across the three tables and practice reshaping between long and wide formats.

### Merging tables with keys (R)

In [ ]:
%%R
library(dplyr)

# Merge enrollment with contract-level info
ga_enroll_contract <- ga_enrollment_checked |>
  left_join(
    ga_contract,
    by = c("contract_id", "year")
  )

# Add service area info
ga_enroll_full <- ga_enroll_contract |>
  left_join(
    ga_service,
    by = c("contract_id", "plan_id", "county", "year")
  )

glue::glue("Rows before merge: {nrow(ga_enrollment_checked)}, after merge: {nrow(ga_enroll_full)}")


### Merging tables with keys (Python)

In [ ]:
# Merge enrollment with contract-level info
ga_enroll_contract = ga_enrollment_checked.merge(
    ga_contract,
    on=["contract_id", "year"],
    how="left"
)

# Add service area info
ga_enroll_full = ga_enroll_contract.merge(
    ga_service,
    on=["contract_id", "plan_id", "county", "year"],
    how="left"
)

ga_enroll_full.shape


### Stacking (binding) multiple datasets

Here we illustrate the idea of stacking using different years within the same file. In practice, you might have one file per year and stack them.

#### R

In [ ]:
%%R
library(dplyr)
library(purrr)

years <- sort(unique(ga_enrollment$year))[1:4]

ga_enrollment_multi <- map_dfr(
  years,
  ~ ga_enrollment |>
      filter(year == .x) |>
      mutate(sample_year = .x)
)

ga_enrollment_multi |>
  count(sample_year)


#### Python

In [ ]:
years = sorted(ga_enrollment["year"].unique())[:4]

frames = []
for y in years:
    df_y = ga_enrollment[ga_enrollment["year"] == y].copy()
    df_y["sample_year"] = y
    frames.append(df_y)

ga_enrollment_multi = pd.concat(frames, ignore_index=True)

ga_enrollment_multi["sample_year"].value_counts().sort_index()


### Reshaping: wide vs long (R)

In [ ]:
%%R
library(dplyr)
library(tidyr)

# Aggregate enrollment by contract and year
ga_contract_year <- ga_enrollment_checked |>
  group_by(contract_id, year) |>
  summarise(
    total_enrollment = sum(enrollment, na.rm = TRUE),
    .groups = "drop"
  )

# Long → wide: years as columns
ga_contract_wide <- ga_contract_year |>
  pivot_wider(
    names_from = year,
    values_from = total_enrollment,
    names_prefix = "enroll_"
  )

# Wide → long: back to year/enrollment columns
ga_contract_long <- ga_contract_wide |>
  pivot_longer(
    cols = starts_with("enroll_"),
    names_to = "year",
    names_prefix = "enroll_",
    values_to = "total_enrollment"
  ) |>
  mutate(year = as.integer(year))

glimpse(ga_contract_wide)
glimpse(ga_contract_long)


### Reshaping: wide vs long (Python)

In [ ]:
# Aggregate enrollment by contract and year
ga_contract_year = (
    ga_enrollment_checked
      .groupby(["contract_id", "year"], as_index=False)
      .agg(total_enrollment=("enrollment", "sum"))
)

# Long → wide: years as columns
ga_contract_wide = ga_contract_year.pivot(
    index="contract_id",
    columns="year",
    values="total_enrollment"
).add_prefix("enroll_").reset_index()

# Wide → long: back to year/enrollment columns
ga_contract_long = ga_contract_wide.melt(
    id_vars="contract_id",
    var_name="year",
    value_name="total_enrollment"
)

ga_contract_long["year"] = (
    ga_contract_long["year"].str.replace("enroll_", "", regex=False).astype(int)
)

ga_contract_wide.head(), ga_contract_long.head()


# Saving cleaned data and output

Finally, we show how to save cleaned datasets and summaries so you can reuse them in future notebooks or scripts.

### Saving cleaned data to CSV (R)

In [ ]:
%%R
library(readr)

write_csv(
  ga_enrollment_checked,
  "ga-enrollment-clean.csv"
)


### Saving cleaned data to CSV (Python)

In [ ]:
ga_enrollment_checked.to_csv(
    "ga-enrollment-clean.csv",
    index=False
)


### Saving native formats and summaries

In a real project, you might also save native R/Python formats (`.rds`, `.pkl`) and summary tables used in reports. Here we just illustrate a simple summary and write it out as CSV.

#### R

In [ ]:
%%R
enroll_summary <- ga_enrollment_checked |>
  group_by(year, rural) |>
  summarise(
    avg_enrollment = mean(enrollment, na.rm = TRUE),
    n_plans        = n(),
    .groups        = "drop"
  )

write_csv(
  enroll_summary,
  "enroll-summary-by-year-rural.csv"
)


#### Python

In [ ]:
enroll_summary = (
    ga_enrollment_checked
      .groupby(["year", "rural"], as_index=False)
      .agg(
          avg_enrollment=("enrollment", "mean"),
          n_plans=("enrollment", "size")
      )
)

enroll_summary.to_csv(
    "enroll-summary-by-year-rural.csv",
    index=False
)
